## Задание

Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (datasets).
Получите качество на тестовой выборке не ниже 88%

Инструкция по выполнению задания

Скачайте тренировочную и тестовою часть датасета
Постройте модель, выбрав стартовую архитектуру
Обучите модель и сверьте качество на тестовой части с заданным порогом
Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.
Формат сдачи работы

Прикрепите ссылку на готовое решение в личном кабинете. Работу можно отправлять в виде ссылки на python-ноутбук из GitHub, Google Colaboratory или аналогичных платформ. Не забудьте открыть доступ на просмотр и комментирование.

Критерии оценки

По итогу выполнения задания вы получите зачёт.

Задание считается выполненным, если:

модель обучается
модель показала качество на тесте более 88%
Задание будет отправлено на доработку, если:

модель вообще не обучилась
качество на тесте ниже порога

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn

# Загрузка данных
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Определение модели
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Обучение модели
def train(model, optimizer, criterion, train_loader):
    train_loss = 0
    train_correct = 0
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader.dataset)
    train_acc = 100. * train_correct / len(train_loader.dataset)
    return train_loss, train_acc

# Оценка модели на тестовом наборе данных
def test(model, criterion, test_loader):
    test_loss = 0
    test_correct = 0
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            test_correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * test_correct / len(test_loader.dataset)
    return test_loss, test_acc

# Обучение модели и оценка ее точности на тестовом наборе данных
model = SimpleNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    train_loss, train_acc = train(model, optimizer, criterion, train_loader)
    test_loss, test_acc = test(model, criterion, test_loader)
    print('Epoch: {} \tTraining Loss: {:.6f} \tTraining Accuracy: {:.2f}% \tTest Loss: {:.6f} \tTest Accuracy: {:.2f}%'.format(
        epoch+1, train_loss, train_acc, test_loss, test_acc))

    if test_acc > 88:
        break

# Добавление слоя и dropout
class ComplexNet(nn.Module):
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Обучение модели и оценка ее точности на тестовом наборе данных
model = ComplexNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(20):
    train_loss, train_acc = train(model, optimizer, criterion, train_loader)
    test_loss, test_acc = test(model, criterion, test_loader)
    print('Epoch: {} \tTraining Loss: {:.6f} \tTraining Accuracy: {:.2f}% \tTest Loss: {:.6f} \tTest Accuracy: {:.2f}%'.format(
        epoch+1, train_loss, train_acc, test_loss, test_acc))

    if test_acc > 88:
        break


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch: 1 	Training Loss: 0.007843 	Training Accuracy: 81.95% 	Test Loss: 0.007165 	Test Accuracy: 83.31%
Epoch: 2 	Training Loss: 0.005981 	Training Accuracy: 86.11% 	Test Loss: 0.006561 	Test Accuracy: 84.54%
Epoch: 3 	Training Loss: 0.005430 	Training Accuracy: 87.28% 	Test Loss: 0.005749 	Test Accuracy: 86.54%
Epoch: 4 	Training Loss: 0.005033 	Training Accuracy: 88.11% 	Test Loss: 0.005657 	Test Accuracy: 86.96%
Epoch: 5 	Training Loss: 0.004714 	Training Accuracy: 88.73% 	Test Loss: 0.005745 	Test Accuracy: 86.72%
Epoch: 6 	Training Loss: 0.004491 	Training Accuracy: 89.46% 	Test Loss: 0.005692 	Test Accuracy: 87.02%
Epoch: 7 	Training Loss: 0.004256 	Training Accuracy: 89.87% 	Test Loss: 0.005580 	Test Accuracy: 87.65%
Epoch: 8 	Training Loss: 0.004097 	Training Accuracy: 90.17% 	Test Loss: 0.005497 	Test Accuracy: 87.80%
Epoch: 9 	Training Loss: 0.003923 	Training Accuracy: 90.71% 	Test Loss

Заданное качество на тестовой выборке не ниже 88% - получено.